Table of Contents
- ASK
    1. key question
    2. Define task

- PREPARE
    1. ROCCC info about data
    2. Few information about dataset

- PROCESS
    1. Import files
    2. Build one df for all year 2022
    3. Clean data
    4. Change dtypes for more optimal
    3. Create new columns
    6. Delete redundant columns


ASK

Key question:

- "How do annual members and casual riders use Cyclistic bikes differently?"

Task:

- Finding differences between casual riders and annual members as well as how they are using bikes.


PREPARE

This data is: 
- Reliable - contains informations about all riders 
- Original - data has been downloaded from original site
- Comprehensive - data have enough records and are completed
- Current - data contains information from previous year
- not Cited - because our data are about fake company, but it is not a problem 

Few information about dataset:
- Data is organized in csv files. One for each month. Therefore we had to downloaded 12 files to colect all data about year 2022.
- Data has been downloaded to folder data/raw and there has been extracted. 
- Each file contains the same columns and has thousands of rows
- Each row contains information about single ride
- Dataset has 13 columns
- Not all rides has start and end station
- Data does't contain information like riderID so we can't look for example how casual rider become member.

PROCESS

Imports

In [1]:
import os
import sys
import pandas as pd
import numpy as np
sys.path.append("..")
from Smaller_Projects.Basic_Data_Cleaning_Helper.Basic_Data_Cleaning_Helper import DataCleaner


In [2]:
# rename one file to the same naming style as others
if os.path.isfile("data/raw/202209-divvy-publictripdata.csv"):
    os.rename("data/raw/202209-divvy-publictripdata.csv", "data/raw/202209-divvy-tripdata.csv")

Test memory usege for one file and explore data

In [3]:
df = pd.read_csv("data/raw/202209-divvy-tripdata.csv")

In [4]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,5156990AC19CA285,electric_bike,2022-09-01 08:36:22,2022-09-01 08:39:05,NaN,NaN,California Ave & Milwaukee Ave,13084,41.93,-87.69,41.922695,-87.697153,casual
1,E12D4A16BF51C274,electric_bike,2022-09-01 17:11:29,2022-09-01 17:14:45,NaN,NaN,NaN,NaN,41.87,-87.62,41.870000,-87.620000,casual
2,A02B53CD7DB72DD7,electric_bike,2022-09-01 17:15:50,2022-09-01 17:16:12,NaN,NaN,NaN,NaN,41.87,-87.62,41.870000,-87.620000,casual
3,C82E05FEE872DF11,electric_bike,2022-09-01 09:00:28,2022-09-01 09:10:32,NaN,NaN,NaN,NaN,41.93,-87.69,41.940000,-87.670000,casual
4,4DEEB4550A266AE1,electric_bike,2022-09-01 07:30:11,2022-09-01 07:32:36,NaN,NaN,NaN,NaN,41.92,-87.73,41.920000,-87.730000,casual


In [5]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701339 entries, 0 to 701338
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             701339 non-null  object 
 1   rideable_type       701339 non-null  object 
 2   started_at          701339 non-null  object 
 3   ended_at            701339 non-null  object 
 4   start_station_name  597559 non-null  object 
 5   start_station_id    597559 non-null  object 
 6   end_station_name    590154 non-null  object 
 7   end_station_id      590154 non-null  object 
 8   start_lat           701339 non-null  float64
 9   start_lng           701339 non-null  float64
 10  end_lat             700627 non-null  float64
 11  end_lng             700627 non-null  float64
 12  member_casual       701339 non-null  object 
dtypes: float64(4), object(9)
memory usage: 439.5 MB


In [6]:
DataCleaner.check_category_type(df)

If category type is good for columns:
                Column     Type  Unique_Count  Total_Count  Is_Category_Good
0              ride_id   object        701339       701339             False
1        rideable_type   object             3       701339              True
2           started_at   object        574082       701339             False
3             ended_at   object        575107       701339             False
4   start_station_name   object          1241       701339              True
5     start_station_id   object          1130       701339              True
6     end_station_name   object          1248       701339              True
7       end_station_id   object          1135       701339              True
8            start_lat  float64        194717       701339             False
9            start_lng  float64        188473       701339             False
10             end_lat  float64          1073       701339             False
11             end_lng  float64       

In [7]:
DataCleaner.null_info(df)

Number of nulls in columns:
                   Nulls_Count Nulls_percent
ride_id                      0           0.0
rideable_type                0           0.0
started_at                   0           0.0
ended_at                     0           0.0
start_station_name      103780          0.17
start_station_id        103780          0.17
end_station_name        111185          0.19
end_station_id          111185          0.19
start_lat                    0           0.0
start_lng                    0           0.0
end_lat                    712           0.0
end_lng                    712           0.0
member_casual                0           0.0


In [8]:
DataCleaner.numeric_info(df)

Numeric columns informations:
      Column   mean  median   std    max    min
0  start_lat  41.90   41.90  0.05  42.07  41.65
1  start_lng -87.65  -87.65  0.03 -87.53 -87.84
2    end_lat  41.90   41.90  0.05  42.15  41.55
3    end_lng -87.65  -87.65  0.03 -87.30 -87.92


In [9]:

# update columns type for datetimes
df["started_at"] = pd.to_datetime(df["started_at"])
df["ended_at"] = pd.to_datetime(df["ended_at"])

# drop index column because its not usefull and use much space
df = df.drop("ride_id",axis=1,errors="ignore")

# udpate last column to binary dtype, member = 0, casual = 1
# udpate last column to binary dtype, member = 1, casual = 0
df["member_casual"] = df["member_casual"] == "member"

# udpate dtype for categorical columns
DataCleaner.change_types_to_category(df)


In [10]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701339 entries, 0 to 701338
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   rideable_type       701339 non-null  category      
 1   started_at          701339 non-null  datetime64[ns]
 2   ended_at            701339 non-null  datetime64[ns]
 3   start_station_name  597559 non-null  category      
 4   start_station_id    597559 non-null  category      
 5   end_station_name    590154 non-null  category      
 6   end_station_id      590154 non-null  category      
 7   start_lat           701339 non-null  float64       
 8   start_lng           701339 non-null  float64       
 9   end_lat             700627 non-null  float64       
 10  end_lng             700627 non-null  float64       
 11  member_casual       701339 non-null  bool          
dtypes: bool(1), category(5), datetime64[ns](2), float64(4)
memory usage: 39.3 MB


We can save about 90% of memory with some transformation. Probably more if we delete lattitude and longtitude data later.

Load all data and save info one DataFrame

In [11]:
df = pd.DataFrame()
col_names = list(pd.read_csv("data/raw/202201-divvy-tripdata.csv", nrows =1))

for month in range(1,13):
    df_month = pd.read_csv(f"data/raw/2022{month:02}-divvy-tripdata.csv", 
            usecols=[i for i in col_names if i != 'ride_id'])

    # udpate last column to binary dtype, member = True, casual = False
    df_month["member_casual"] = df_month["member_casual"] == "member"

    # udpate dtype for categorical columns
    DataCleaner.change_types_to_category(df_month)
    
    df = pd.concat([df,df_month], ignore_index=True)
    
# update columns type for datetimes   
df["started_at"] = pd.to_datetime(df["started_at"])
df["ended_at"] = pd.to_datetime(df["ended_at"]) 
df = df.rename(columns={"member_casual": "is_member"}) 
DataCleaner.change_types_to_category(df)   
df.reindex()   
df.info(memory_usage="deep")    



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5667717 entries, 0 to 5667716
Data columns (total 12 columns):
 #   Column              Dtype         
---  ------              -----         
 0   rideable_type       category      
 1   started_at          datetime64[ns]
 2   ended_at            datetime64[ns]
 3   start_station_name  category      
 4   start_station_id    category      
 5   end_station_name    category      
 6   end_station_id      category      
 7   start_lat           float64       
 8   start_lng           float64       
 9   end_lat             float64       
 10  end_lng             float64       
 11  is_member           bool          
dtypes: bool(1), category(5), datetime64[ns](2), float64(4)
memory usage: 314.1 MB


Dropping  duplicated rows

In [12]:
df = df.drop_duplicates()

In [13]:
DataCleaner.null_info(df)  

Number of nulls in columns:
                   Nulls_Count Nulls_percent
rideable_type                0           0.0
started_at                   0           0.0
ended_at                     0           0.0
start_station_name      833063          0.17
start_station_id        833063          0.17
end_station_name        892740          0.19
end_station_id          892740          0.19
start_lat                    0           0.0
start_lng                    0           0.0
end_lat                   5857           0.0
end_lng                   5857           0.0
is_member                    0           0.0


Dropping  rows containing empty end_lng and end_lat

In [14]:
df = df.dropna(subset=['end_lat'])

In [15]:
DataCleaner.outliers_info(df)  

Info about outliers:
      Column     Type   Mean   Std  Outliers_upper  Outliers_lower
0  start_lat  float64  41.90  0.05          264368          378586
1  start_lng  float64 -87.65  0.03          268825          393395
2    end_lat  float64  41.90  0.07           86349          300808
3    end_lng  float64 -87.65  0.11              12            1596


In [16]:
df["rideable_type"].unique()

['electric_bike', 'classic_bike', 'docked_bike']
Categories (3, object): ['classic_bike', 'docked_bike', 'electric_bike']

In [17]:
DataCleaner.numeric_info(df)

Numeric columns informations:
      Column   mean  median   std    max    min
0  start_lat  41.90   41.90  0.05  45.64  41.64
1  start_lng -87.65  -87.64  0.03 -73.80 -87.84
2    end_lat  41.90   41.90  0.07  42.37   0.00
3    end_lng -87.65  -87.64  0.11   0.00 -88.14


In [18]:
percent_of_members = round(sum(df["is_member"]==True) / len(df),2)
print(f"Members rides percent: {percent_of_members}\nCasuals rides percent: {round(1-percent_of_members,2)}")

Members rides percent: 0.59
Casuals rides percent: 0.41


New columns

Add duration column for duration of ride (in minutes)

In [19]:
df["duration"] = ((df["ended_at"] - df["started_at"]).dt.total_seconds()/60).astype(np.float32)

Drop rides that takes less then 30s

In [20]:
df = df[df["duration"] > 0.5].dropna()

Add distance column for distance of ride (in meters)

In [21]:
def calculate_distance(lon1, lat1, lon2, lat2):

    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    m = 6371000 * c
    return m

In [22]:
df["distance"] = calculate_distance(df['start_lng'],df['start_lat'],df['end_lng'],df['end_lat']).astype(np.float32)

In [23]:
#now we dont need gps columns se we can drop them
df = df.drop(["start_lng","start_lat","end_lng","end_lat"],axis=1)

Add time column for exact time of stated ride (in float hours)

In [24]:
df["started_time"] = (df["started_at"].dt.hour +df["started_at"].dt.minute/60 +df["started_at"].dt.second/3600).astype(np.float16)

Add week_day column for day of the week 

In [25]:
df["week_day"] = df["started_at"].dt.dayofweek.astype(np.int8)

Add month column 

In [26]:
df["month"] = df["started_at"].dt.month.astype(np.int8)

Add day_of_year column 

In [27]:
df["day_of_year"] = df["started_at"].dt.day_of_year.astype(np.uint16)

Add day column 

In [28]:
df["day"] = df["started_at"].dt.day.astype(np.int8)

In [29]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4320790 entries, 0 to 5667716
Data columns (total 15 columns):
 #   Column              Dtype         
---  ------              -----         
 0   rideable_type       category      
 1   started_at          datetime64[ns]
 2   ended_at            datetime64[ns]
 3   start_station_name  category      
 4   start_station_id    category      
 5   end_station_name    category      
 6   end_station_id      category      
 7   is_member           bool          
 8   duration            float32       
 9   distance            float32       
 10  started_time        float16       
 11  week_day            int8          
 12  month               int8          
 13  day_of_year         uint16        
 14  day                 int8          
dtypes: bool(1), category(5), datetime64[ns](2), float16(1), float32(2), int8(3), uint16(1)
memory usage: 202.5 MB


In [30]:
df.head()

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,is_member,duration,distance,started_time,week_day,month,day_of_year,day
0,electric_bike,2022-01-13 11:59:47,2022-01-13 12:02:44,Glenwood Ave & Touhy Ave,525,Clark St & Touhy Ave,RP-007,False,2.950000,699.546753,12.000000,3,1,13,13
1,electric_bike,2022-01-10 08:41:56,2022-01-10 08:46:17,Glenwood Ave & Touhy Ave,525,Clark St & Touhy Ave,RP-007,False,4.350000,694.321716,8.695312,0,1,10,10
2,classic_bike,2022-01-25 04:53:40,2022-01-25 04:58:01,Sheffield Ave & Fullerton Ave,TA1306000016,Greenview Ave & Fullerton Ave,TA1307000001,True,4.350000,1000.832275,4.894531,1,1,25,25
3,classic_bike,2022-01-04 00:18:04,2022-01-04 00:33:00,Clark St & Bryn Mawr Ave,KA1504000151,Paulina St & Montrose Ave,TA1309000021,False,14.933333,2462.779297,0.301025,1,1,4,4
4,classic_bike,2022-01-20 01:31:10,2022-01-20 01:37:12,Michigan Ave & Jackson Blvd,TA1309000002,State St & Randolph St,TA1305000029,True,6.033333,814.537720,1.519531,3,1,20,20


Export prepered data

In [31]:
df.to_parquet("data/preprocessed/dataset.parquet")

Export to csv if needed, but take long time and 4 times more space

In [32]:
#df.to_csv("data/preprocessed/dataset.csv")